In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
data_dir = '../data/'
train_dir = data_dir + 'images/train/'
valid_dir = data_dir + 'images/valid/'

transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.RandomRotation(10),
    v2.RandomCrop(224),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_set = ImageFolder(train_dir, transform=transform, target_transform=None)
valid_set = ImageFolder(valid_dir, transform=transform, target_transform=None)

print(train_set.class_to_idx)

fig, ax = plt.subplots(figsize=(12,8), ncols=4)
for i in range(4):
    image, label = train_set[i]

    # change dimensions to matplotlib order, denormalize and clip RGB values to [0,1]
    rgb_img = (image.permute(1, 2, 0).numpy() * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])).clip(0, 1)
    
    ax[i].imshow(rgb_img)